In [1]:
# Copyright 2019, The TensorFlow Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""Training a CNN on MNIST in TF Eager mode with DP-SGD optimizer."""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from absl import app
from absl import flags

import numpy as np
import tensorflow.compat.v1 as tf

from tensorflow_privacy.privacy.analysis.rdp_accountant import compute_rdp
from tensorflow_privacy.privacy.analysis.rdp_accountant import get_privacy_spent
from tensorflow_privacy.privacy.optimizers.dp_optimizer import DPGradientDescentGaussianOptimizer

GradientDescentOptimizer = tf.train.GradientDescentOptimizer
tf.enable_eager_execution()

dpsgd = True
learning_rate = 0.15
noise_multiplier = 1.1
l2_norm_clip = 1.0
batch_size = 250
epochs = 60
microbatches =250

def compute_epsilon(steps):
  """Computes epsilon value for given hyperparameters."""
  if noise_multiplier == 0.0:
    return float('inf')
  orders = [1 + x / 10. for x in range(1, 100)] + list(range(12, 64))
  sampling_probability = batch_size / 60000
  rdp = compute_rdp(q=sampling_probability,
                    noise_multiplier=noise_multiplier,
                    steps=steps,
                    orders=orders)
  # Delta is set to 1e-5 because MNIST has 60000 training points.
  return get_privacy_spent(orders, rdp, target_delta=1e-5)[0]

In [ ]:
  if dpsgd and batch_size % microbatches != 0:
    raise ValueError('Number of microbatches should divide evenly batch_size')

  # Fetch the mnist data
  train, test = tf.keras.datasets.mnist.load_data()
  train_images, train_labels = train
  test_images, test_labels = test

  # Create a dataset object and batch for the training data
  dataset = tf.data.Dataset.from_tensor_slices(
      (tf.cast(train_images[..., tf.newaxis]/255, tf.float32),
       tf.cast(train_labels, tf.int64)))
  dataset = dataset.shuffle(1000).batch(batch_size)

  # Create a dataset object and batch for the test data
  eval_dataset = tf.data.Dataset.from_tensor_slices(
      (tf.cast(test_images[..., tf.newaxis]/255, tf.float32),
       tf.cast(test_labels, tf.int64)))
  eval_dataset = eval_dataset.batch(10000)

  # Define the model using tf.keras.layers
  mnist_model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(16, 8,
                             strides=2,
                             padding='same',
                             activation='relu'),
      tf.keras.layers.MaxPool2D(2, 1),
      tf.keras.layers.Conv2D(32, 4, strides=2, activation='relu'),
      tf.keras.layers.MaxPool2D(2, 1),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(32, activation='relu'),
      tf.keras.layers.Dense(10)
  ])

In [2]:
  # Instantiate the optimizer
  if dpsgd:
    opt = DPGradientDescentGaussianOptimizer(
        l2_norm_clip=l2_norm_clip,
        noise_multiplier=noise_multiplier,
        num_microbatches=microbatches,
        learning_rate=learning_rate)
  else:
    opt = GradientDescentOptimizer(learning_rate=learning_rate)

  # Training loop.
  steps_per_epoch = 60000 // batch_size
  for epoch in range(epochs):
    # Train the model for one epoch.
    for (_, (images, labels)) in enumerate(dataset.take(-1)):
      with tf.GradientTape(persistent=True) as gradient_tape:
        # This dummy call is needed to obtain the var list.
        logits = mnist_model(images, training=True)
        var_list = mnist_model.trainable_variables

        # In Eager mode, the optimizer takes a function that returns the loss.
        def loss_fn():
          logits = mnist_model(images, training=True)  # pylint: disable=undefined-loop-variable,cell-var-from-loop
          loss = tf.nn.sparse_softmax_cross_entropy_with_logits(
              labels=labels, logits=logits)  # pylint: disable=undefined-loop-variable,cell-var-from-loop
          # If training without privacy, the loss is a scalar not a vector.
          if not dpsgd:
            loss = tf.reduce_mean(input_tensor=loss)
          return loss

        if dpsgd:
          grads_and_vars = opt.compute_gradients(loss_fn, var_list,
                                                 gradient_tape=gradient_tape)
        else:
          grads_and_vars = opt.compute_gradients(loss_fn, var_list)
      print(grads_and_vars)
      opt.apply_gradients(grads_and_vars)

    # Evaluate the model and print results
    for (_, (images, labels)) in enumerate(eval_dataset.take(-1)):
      logits = mnist_model(images, training=False)
      correct_preds = tf.equal(tf.argmax(input=logits, axis=1), labels)
    test_accuracy = np.mean(correct_preds.numpy())
    print('Test accuracy after epoch %d is: %.3f' % (epoch, test_accuracy))

    # Compute the privacy budget expended so far.
    if dpsgd:
      eps = compute_epsilon((epoch + 1) * steps_per_epoch)
      print('For delta=1e-5, the current epsilon is: %.2f' % eps)
    else:
      print('Trained with vanilla non-private SGD optimizer')

KeyboardInterrupt: 

In [4]:
      with tf.GradientTape(persistent=True) as gradient_tape:
        # This dummy call is needed to obtain the var list.
        logits = mnist_model(images, training=True)
        var_list = mnist_model.trainable_variables

        # In Eager mode, the optimizer takes a function that returns the loss.
        def loss_fn():
          logits = mnist_model(images, training=True)  # pylint: disable=undefined-loop-variable,cell-var-from-loop
          loss = tf.nn.sparse_softmax_cross_entropy_with_logits(
              labels=labels, logits=logits)  # pylint: disable=undefined-loop-variable,cell-var-from-loop
          # If training without privacy, the loss is a scalar not a vector.
          if not dpsgd:
            loss = tf.reduce_mean(input_tensor=loss)
          return loss

        if dpsgd:
          grads_and_vars = opt.compute_gradients(loss_fn, var_list,
                                                 gradient_tape=gradient_tape)
        else:
          grads_and_vars = opt.compute_gradients(loss_fn, var_list)

In [17]:
grads_and_vars[0]

(<tf.Tensor: shape=(8, 8, 1, 16), dtype=float32, numpy=
 array([[[[ 6.22760970e-03,  3.78497690e-03,  2.89283018e-03, ...,
           -3.95402825e-03, -3.61379376e-03,  5.04491478e-03]],
 
         [[-1.71382457e-03,  1.08591001e-03,  4.03887639e-03, ...,
            7.12561305e-04,  3.86093417e-03, -1.03019807e-03]],
 
         [[ 2.61681154e-03, -1.30665474e-04, -3.78490542e-03, ...,
            6.87710941e-03,  6.90065930e-03,  3.73602874e-04]],
 
         ...,
 
         [[ 3.61003826e-04,  3.26182903e-03,  6.40055398e-04, ...,
           -5.99179766e-04,  4.15064453e-04, -2.40638779e-04]],
 
         [[-6.92460744e-04, -1.73666410e-03, -2.15681852e-03, ...,
           -2.28105788e-03, -3.94116063e-03,  1.82628026e-03]],
 
         [[-8.67268350e-03,  3.69376037e-03, -6.49256574e-04, ...,
           -4.26838826e-03, -4.20047948e-03,  6.07611658e-03]]],
 
 
        [[[-1.05375731e-02,  5.85369021e-03, -3.77016980e-03, ...,
            2.36617750e-03, -6.99667493e-03,  5.35180792e-04